In [1]:
import theano
import theano.tensor as tt

import numpy as np

from theano_ops.celerite.factor import FactorOp
from theano_ops.celerite.solve import SolveOp

In [2]:
a = tt.dvector()
U = tt.dmatrix()
V = tt.dmatrix()
P = tt.dmatrix()
Y = tt.dmatrix()

factor_op = FactorOp(J=2)
d, W, _ = factor_op(a, U, V, P)
solve_op = SolveOp(J=2, n_rhs=1)
Z, F, G = solve_op(U, P, d, W, Y)

f = theano.function([a, U, V, P, Y], Z)
g = theano.function([a, U, V, P, Y], theano.grad(tt.sum(Z), [a, U, V, P, Y]))

In [6]:
N = 10000
diag = np.random.rand(N)
x = np.sort(np.random.uniform(0, 10, N))
y = np.sin(x)[:, None]

a_real = np.random.rand(2)
c_real = np.random.rand(2)
a_comp = np.array([])
b_comp = np.array([])
c_comp = np.array([])
d_comp = np.array([])

a_ = diag + np.sum(a_real) + np.sum(a_comp)

U_ = np.concatenate((
    a_real[None, :] + np.zeros_like(x)[:, None],
    a_comp[None, :] * np.cos(d_comp[None, :] * x[:, None])
    + b_comp[None, :] * np.sin(d_comp[None, :] * x[:, None]),
    a_comp[None, :] * np.sin(d_comp[None, :] * x[:, None])
    - b_comp[None, :] * np.cos(d_comp[None, :] * x[:, None]),
), axis=1)

V_ = np.concatenate((
    np.zeros_like(a_real)[None, :] + np.ones_like(x)[:, None],
    np.cos(d_comp[None, :] * x[:, None]),
    np.sin(d_comp[None, :] * x[:, None]),
), axis=1)

dx = x[1:] - x[:-1]
P_ = np.concatenate((
    np.exp(-c_real[None, :] * dx[:, None]),
    np.exp(-c_comp[None, :] * dx[:, None]),
    np.exp(-c_comp[None, :] * dx[:, None]),
), axis=1)

%timeit f(a_, U_, V_, P_, y)
%timeit g(a_, U_, V_, P_, y)

1000 loops, best of 3: 387 µs per loop
1000 loops, best of 3: 950 µs per loop
